## Libraries


In [ ]:
# !pip install -r ~/code/benitomartin/FoodScore/requirements.txt

In [1]:
import os
import cv2
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers 
from tensorflow.keras import Model 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import load_img, img_to_array, to_categorical, image_dataset_from_directory
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from tensorflow.keras import losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanIoU


import pickle

## Data import

In [2]:
coord = pd.DataFrame()

for i in range(1, 101, 1):
    path = f"../raw_data/UECFOOD100/{i}"
    data = pd.read_csv(f"{path}/bb_info.txt", sep=' ', header=0, index_col="img")
    data_df = pd.DataFrame(data)
    data_df["label"] = i
    coord = pd.concat([coord, data_df])
    


In [3]:
coord = coord.reset_index()

### DataFrame with label and coordinates

In [4]:
coord.head()

,img,x1,y1,x2,y2,label
0,1,0,143,370,486,1
1,2,20,208,582,559,1
2,3,2,110,243,410,1
3,4,0,237,286,536,1
4,5,8,28,761,585,1


In [5]:
coord = coord.rename(columns={"img": "img_name"})

In [6]:
coord.head()

,img_name,x1,y1,x2,y2,label
0,1,0,143,370,486,1
1,2,20,208,582,559,1
2,3,2,110,243,410,1
3,4,0,237,286,536,1
4,5,8,28,761,585,1


### Rescaling and Normalization

In [7]:
# function to normalize bounding box

def normalize_bbox(row):
    # Read in the image and get its dimensions
    image_path = f"../raw_data/UECFOOD100/{(row['label'])}/{(row['img_name'])}.jpg"
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    
    # Normalize the coordinates
    x1_norm = row['x1'] / width
    y1_norm = row['y1'] / height
    x2_norm = row['x2'] / width
    y2_norm = row['y2'] / height
    
    # Return normalized coordinates
    return pd.Series({'x1_norm': x1_norm, 'y1_norm': y1_norm, 'x2_norm': x2_norm, 'y2_norm': y2_norm})

# Apply the normalize_bbox function to each row of the DataFrame
normalized_bbox_df = coord.apply(normalize_bbox, axis=1)

# Concatenate the original DataFrame with the new normalized DataFrame
rescaled_coord = pd.concat([coord, normalized_bbox_df], axis=1).drop(columns=['x1', 'y1','x2','y2'])


In [8]:
rescaled_coord.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm
0,1,1,0.0000,0.238333,0.46250,0.810000
1,2,1,0.0250,0.346667,0.72750,0.931667
2,3,1,0.0025,0.183333,0.30375,0.683333
3,4,1,0.0000,0.395000,0.35750,0.893333
4,5,1,0.0100,0.046667,0.95125,0.975000


### add image paths

In [9]:
list_paths = [f"../raw_data/UECFOOD100/{int(row['label'])}/{int(row['img_name'])}.jpg" for _, row in coord.iterrows()]


In [10]:
rescaled_coord["paths"] = pd.DataFrame(list_paths).copy()

In [11]:
rescaled_coord.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm,paths
0,1,1,0.0000,0.238333,0.46250,0.810000,../raw_data/UECFOOD100/1/1.jpg
1,2,1,0.0250,0.346667,0.72750,0.931667,../raw_data/UECFOOD100/1/2.jpg
2,3,1,0.0025,0.183333,0.30375,0.683333,../raw_data/UECFOOD100/1/3.jpg
3,4,1,0.0000,0.395000,0.35750,0.893333,../raw_data/UECFOOD100/1/4.jpg
4,5,1,0.0100,0.046667,0.95125,0.975000,../raw_data/UECFOOD100/1/5.jpg


### balancing Dataset

In [13]:
coord.shape

(14611, 6)

In [14]:
def rebalancing(df: pd.DataFrame, classes: list, av_number: int = 110, random_state: int = 1) -> pd.DataFrame:
    df_new = df.copy()
    for class_ in classes:
        class_df = df_new[df_new['label'] == class_]
        class_count = len(class_df)
        if class_count > av_number:
            drop_indices = np.random.choice(class_df.index, class_count - av_number, replace=False)
            df_new = df_new.drop(drop_indices)
        else:
            pass
    return df_new

In [15]:
classes = list(set(rescaled_coord.label))

In [16]:
df = rebalancing(rescaled_coord, classes, av_number= 50, random_state=1)

In [17]:
rescaled_coord[rescaled_coord['label']==100].shape

(104, 7)

In [18]:
df[df['label']==100].shape

(50, 7)

### load downscaled pictures into array

In [19]:
from tqdm.auto import tqdm

/Users/katharinarolfs/.pyenv/versions/3.10.6/envs/FoodScore/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
df.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm,paths
0,1,1,0.000,0.238333,0.4625,0.810000,../raw_data/UECFOOD100/1/1.jpg
12,14,1,0.000,0.561667,0.3000,1.000000,../raw_data/UECFOOD100/1/14.jpg
20,24,1,0.000,0.266910,1.0000,1.000000,../raw_data/UECFOOD100/1/24.jpg
24,28,1,0.022,0.402985,0.3600,0.785075,../raw_data/UECFOOD100/1/28.jpg
30,35,1,0.536,0.346667,0.9740,0.936000,../raw_data/UECFOOD100/1/35.jpg


In [21]:
df_shuffled = df.sample(frac=1, random_state=42)
df_shuffled.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm,paths
5049,2997,31,0.137500,0.012500,0.945312,0.791667,../raw_data/UECFOOD100/31/2997.jpg
8243,5266,52,0.264444,0.000000,1.000000,0.700000,../raw_data/UECFOOD100/52/5266.jpg
8384,5397,54,0.170000,0.012285,0.814000,0.800983,../raw_data/UECFOOD100/54/5397.jpg
3632,2114,22,0.369650,0.000000,1.000000,1.000000,../raw_data/UECFOOD100/22/2114.jpg
2549,1410,15,0.000000,0.000000,0.795556,1.000000,../raw_data/UECFOOD100/15/1410.jpg


In [42]:
df_shuffled.to_csv('df_shuffled.csv')

In [23]:
color_order = "BGR"
dims = (224,224)

images = np.empty((len(df_shuffled), dims[0], dims[1], 3), dtype=np.float32)

for i, path in enumerate(tqdm(df_shuffled.paths.values)):
    img = cv2.imread(path)
    img = cv2.resize(img, dims, interpolation=cv2.INTER_AREA)
    if color_order == "RGB":
        img = img[:,:,::-1]
    images[i, :, :, :] = img/255

100%|██████████████████████████████████████| 5000/5000 [00:23<00:00, 217.08it/s]


In [41]:
np.savez_compressed('imgs_100.npz', images)


In [24]:
labels = np.array(df_shuffled.label)
bboxes = np.array(df_shuffled[['x1_norm','y1_norm','x2_norm','y2_norm']], dtype="float32")
paths = np.array(df_shuffled.paths)

In [25]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [26]:
if len(lb.classes_) == 2:
    print("two classes")
    labels = to_categorical(labels)

In [27]:
len(set(df_shuffled.label))

100

In [28]:
tvImages, testImages,tvLabels, testLabels =\
train_test_split(images,
                 labels,
                 test_size=0.20,
                 random_state=42)

In [29]:
trainImages, valImages,trainLabels, valLabels=\
train_test_split(tvImages,
                 tvLabels,
                 test_size=0.20,
                 random_state=42)

## Model

In [30]:
vgg = VGG16(weights="imagenet",
            include_top=False,
            input_tensor=layers.Input(shape=(224, 224, 3)))


vgg.trainable = False

flatten = vgg.output
flatten = layers.Flatten()(flatten)

softmaxHead = layers.Dense(512, activation="relu")(flatten)
softmaxHead = layers.Dropout(0.5)(softmaxHead)
softmaxHead = layers.Dense(512, activation="relu")(softmaxHead)
softmaxHead = layers.Dropout(0.5)(softmaxHead)
softmaxHead = layers.Dense(len(set(df_shuffled.label)), activation="softmax", name="class_label")(softmaxHead)


In [31]:
model = Model(
    inputs=vgg.input,
    outputs=(softmaxHead))

In [32]:
losses = {
    "class_label": 'categorical_crossentropy',}

In [33]:
lossWeights = {
    "class_label": 1.0,}

In [34]:
trainTargets = {
    "class_label": trainLabels,}

In [35]:
testTargets = {
    "class_label": testLabels,}

In [36]:
valTargets = {
    "class_label": valLabels,}

In [37]:
metrics = {
    "class_label": "categorical_accuracy",}

In [38]:
opt = Adam(0.001)


model.compile(loss=losses, 
              optimizer=opt, 
              metrics=metrics, 
              loss_weights=lossWeights)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [39]:
es = EarlyStopping(monitor = 'loss',
                   patience = 5,
                   verbose = 0,
                   restore_best_weights = True)

In [43]:
from datetime import datetime
import keras
logdir = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [44]:
history_100_label_only = model.fit(
    trainImages,
    trainTargets,
    validation_data=(valImages, valTargets),
    batch_size=32,
    epochs=1000,
    verbose=1,
    callbacks = [es, tensorboard_callback],
    )

Epoch 1/1000
100/100 [==============================] - 419s 4s/step - loss: 4.5880 - categorical_accuracy: 0.0162 - val_loss: 4.5974 - val_categorical_accuracy: 0.0100
Epoch 2/1000
100/100 [==============================] - 422s 4s/step - loss: 4.5491 - categorical_accuracy: 0.0178 - val_loss: 4.5531 - val_categorical_accuracy: 0.0175
Epoch 3/1000
100/100 [==============================] - 417s 4s/step - loss: 4.5363 - categorical_accuracy: 0.0191 - val_loss: 4.4964 - val_categorical_accuracy: 0.0213
Epoch 4/1000
100/100 [==============================] - 412s 4s/step - loss: 4.4975 - categorical_accuracy: 0.0209 - val_loss: 4.5192 - val_categorical_accuracy: 0.0250
Epoch 5/1000
100/100 [==============================] - 416s 4s/step - loss: 4.4789 - categorical_accuracy: 0.0234 - val_loss: 4.5025 - val_categorical_accuracy: 0.0162
Epoch 6/1000
100/100 [==============================] - 414s 4s/step - loss: 4.4506 - categorical_accuracy: 0.0222 - val_loss: 4.4453 - val_categorical_acc

## Save model

In [ ]:
# VGG16

pickle.dump(model, open('vgg16_100classes_50imgs.pkl', 'wb'))